In [1]:
SQL_SOURCE = 'https://ssb602.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-73256dea51c1>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

## Task 1: Familiarize with SQL Clauses

### Sort data by start_station_id, tripduration 
### -Only checking trips with duration <= 3 hours 

In [ ]:
test = '''
SELECT *
FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id, tripduration 
'''

In [ ]:
sorted_trip_data = pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
sorted_trip_data.head(4)

### Only show the top/last 10 records 

In [ ]:
test = '''SELECT * from citibike
limit 10'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

### List all unique start_station_id values

In [ ]:
test = '''select distinct(start_station_id) AS unique_station
from citibike'''

In [ ]:
unique_station_id = pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
unique_station_id['unique_station'].head(5)

### Count the number of trips

In [ ]:
test = '''select count(tripduration) as Number_of_trips from citibike'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

### Find the average/min/max trip duration

In [ ]:
test = '''select max(tripduration) as Max_Trip_Duration,
min(tripduration) as Min_Trip_Duration,
AVG(tripduration) as Avg_Trip_Duration
from citibike'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

## Task 2 — Working with date/time

### Selecting trips started on the weekends 

In [ ]:
test = '''select * from citibike
where extract(DOW from starttime) IN (0,6)'''

In [ ]:
weekend_trips = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
weekend_trips['starttime'].head(5)

### Selecting trips started on Feb-02-2015 only 

In [ ]:
test = '''select * from citibike
where extract(DAY from starttime) in (2) and
extract(MONTH from starttime) in (2) and 
extract(YEAR from starttime) in (2015)'''

In [ ]:
feb2_trips = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
feb2_trips.starttime.head(5)

### Selecting trips started on the weekdays

In [ ]:
test = '''select * from citibike
where extract(DOW from starttime) NOT IN (0,6)'''

In [ ]:
weekday_trips = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
weekday_trips['starttime'].head(5)

### What are average trip duration during weekends?

In [ ]:
test = '''select avg(tripduration) as avg_trip_duration from citibike
where extract(DOW from starttime) IN (0,6)'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

## Task 3 — Working with Space

### Showing the list of start station locations - Using GROUP BY 

In [ ]:
test = '''select start_station_latitude, start_station_longitude from citibike
group by start_station_latitude, start_station_longitude'''

In [ ]:
start_station = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
start_station.head(4)

### Showing the number of trips started per station 

In [ ]:
test = '''select count(*) from citibike
group by start_station_id, start_station_latitude, start_station_longitude'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

### Showing the number of trips started per station 
### ... but only for stations within 500m of Time Square! 

In [ ]:
test = '''select CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude))
as the_geom_webmercator, min(cartodb_id) as cartodb_id
from citibike
where ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,500)
group by start_station_id, start_station_latitude, start_station_longitude
'''

In [ ]:
times_square_trips = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
times_square_trips.head(5)

## Task 4 — Putting it all together

### Find the station that had the longest average trip duration during 
### weekends and within 500m of TimeSquare! 

In [ ]:
test = '''select start_station_id, avg(tripduration) from citibike
where ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,500)
                 group by start_station_id'''

In [ ]:
station_id_timesq_500m = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
max_avg = station_id_timesq_500m['avg'].max()

In [ ]:
station_id_max_avg_trip = station_id_timesq_500m[station_id_timesq_500m['avg'] == max_avg]

In [ ]:
print("Start station id with longest avg trip duration is - "+str(station_id_max_avg_trip['start_station_id']))

### Extra: create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours
### The number of trips per each pair of stations are output as attributes of these lines. 

In [ ]:
test = '''select ST_Makeline(
  CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
  CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude))
  )
as the_geom_webmercator, min(cartodb_id) as cartodb_id
from citibike
where ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,500)
group by start_station_id, start_station_latitude, start_station_longitude,
end_station_id, end_station_latitude, end_station_longitude'''

In [ ]:
mark_lines = pd.read_csv(StringIO.StringIO(queryCartoDB(test)),sep=',')

In [ ]:
mark_lines.head(5)

## ASH Score -100. Extra Credit 0